In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
fn = snakemake.input[0]


dfs = []

In [ ]:
for fn in snakemake.input:
    fn = Path(fn)
    
    year = fn.parts[1].split("_")[0]
    wacc = fn.parts[1].split("_")[1]
    esc = fn.parts[2]
    exporter = fn.parts[3].split("-")[0]
    importer = fn.parts[3].split("-")[1]

    df = pd.read_csv(fn, delimiter=";")

    df.insert(0, "wacc", wacc)
    df.insert(0, "year", year)
    df.insert(0, "importer", importer)
    df.insert(0, "exporter", exporter)
    df.insert(0, "esc", esc)
    
    dfs.append(df)    

In [ ]:
df = pd.concat(dfs)
df = df.set_index(['year','wacc','esc','exporter','importer','category','subcategory'], verify_integrity=True)

In [ ]:
df.to_csv(snakemake.output['results'], sep=";", quotechar="\"")